# T1.1 - Solution
## Authors:
- Leonardo Kaplan 1212509
- Nino Fabrizio Tiriticco Lizardo 1113203

In [ ]:
# Pacotes usados
import pandas as pd # Para pegar os dados dos arquivos
from IPython.display import display # Para mostrar mais de uma informação em uma mesma célula
import ast # Para transformar string/object em estruturas de dados (listas, dicionários, ...)
import numpy as np # Para obter o total de valores por um atributo, se um data frame está vazio
import matplotlib.pyplot as plt # Para plotar gráficos
import matplotlib

In [ ]:
# Carregando dados de cada um dos arquivos
DataQualisRaw = pd.read_excel('Files/Qualis CC 2013-2016.xlsx')
DataDocentesRaw = pd.read_csv('Files/docentes.csv')
DataDiscentesRaw = pd.read_csv('Files/discentes.csv')
DataProducaoRaw = pd.read_csv('Files/producao.csv')
DataTrabalhosRaw = pd.read_csv('Files/trabalhos.csv')

## Análise prévia dos dados:

In [ ]:
# Pela função info, parece que temos todos os dados disponíveis para cada coluna (sem valores nulos)
display(DataQualisRaw.info())

# Pelas 5 primeiras tuplas, os dados parecem não precisar tratamento
DataQualisRaw.head()

# Fazendo uma análise prévia dos dados, DataQualisRaw parece representar o título de papers associado a um código(?)

In [ ]:
# Pela função info, parece que temos todos os dados disponíveis para cada coluna (sem valores nulos)
display(DataDocentesRaw.info())

# Pelas 5 primeiras tuplas, os dados parecem não precisar tratamento
DataDocentesRaw.head()

# Fazendo uma análise prévia dos dados, DataDocentesRaw parece representar docentes e sua relação com a instiuição acadêmica/universidade

In [ ]:
# Pela função info, parece que temos dados faltando para algumas das colunas (há valores nulos)
display(DataDiscentesRaw.info())

# Pelas tabela, devemos precisar tratar os dados da coluna "abrev". Importante ressaltar que alguns dos dados em "orientadores" parecem representar listas de dicionários através de string/object.
DataDiscentesRaw.head()

# Fazendo uma análise prévia dos dados, DataDiscentesRaw parece representar os alunos de pós-graduação das instituições acadêmicas/universidades e todos os dados atrelados a eles.

In [ ]:
# A função info nos mostra que temos valores nulos em algumas das colunas (total de valores não-nulos menor que o total de tuplas da tabela)
display(DataProducaoRaw.info())

pd.set_option('display.max_columns', 35) # Para poder visualizar todas as colunas deste data frame

# As 5 primeiras tuplas da tabela nos mostram que pelo menos alguns dados precisam ser tratados nas colunas:
# - dict_paper_autores (os dados dessa coluna na verdade representam listas de dicionários) [VER PRÓXIMA CÉLULA]
# - paper_autores (os dados dessa coluna na verdade representam listas de strings/objects) [VER CÉLULA PRÓXIMA À SEGUINTE]
# - doi
# - periodico (talvez?)
# - ano (número fracionário para ano faz sentido? cuidado, faz sim se formos considerar como período [ex.: 2018.1 e 2018.2])
DataProducaoRaw.head()

# Fazendo uma análise prévia dos dados, parece que DataProducaoRaw lista papers publicados por integrantes (docentes?) de instituições acadêmicas/universidades, informando vários outros dados ligados a esses papers.

In [ ]:
# Vemos que uma célula da segunda coluna de DataProducaoRaw é uma string/object que representa uma lista de dicionários
display(DataProducaoRaw['dict_paper_autores'][0])

# Transformando uma dessas string/objects em uma lista de dicionários
dictionaryList = ast.literal_eval(DataProducaoRaw['dict_paper_autores'][0])

display(dictionaryList) # Lista de dicionários
display(dictionaryList[0]) # Dicionário
display(dictionaryList[0]['categoria']) # Valor atribuído à chave 'categoria'
dictionaryList[0]['nome'] # Valor atribuído à chave 'nome'

In [ ]:
# Vemos que uma célula da coluna 'paper_autores' de DataProducaoRaw é uma string/object que representa uma lista de strings/objects
display(DataProducaoRaw['paper_autores'][0])

stringList = ast.literal_eval(DataProducaoRaw['paper_autores'][0])

display(stringList) # Lista de strings/objects
display(stringList[0]) # Uma (TADAM!) string/object

In [ ]:
# Pela função info, parece que temos valores nulos em algumas das colunas
display(DataTrabalhosRaw.info())

# Pelas 5 primeiras tuplas, parece que precisaremos tratar os dados das colunas:
# - keywords
# - palavras-chave
# - paginas (número fracionário para páginas faz sentido?)
# - programa (talvez?)
DataTrabalhosRaw.head()

# Fazendo uma análise prévia dos dados, DataTrabalhosRaw parece representar os dados referentes aos trabalhos de conclusão de curso dos alunos de pós-graduação

# Perguntas:
## 1) Quantos professores (docentes) havia em cada instituição em 2017, em cada quadro (permanente, colaborador)?

In [ ]:
# DataDocentesRaw contém os dados necessários para responder a pergunta (colunas "ies" e "categoria")
DataDocentesRaw.head()

In [ ]:
# Verificando quais as instituições, parecem OK
display(DataDocentesRaw['ies'].unique())

# Verificando quais as categorias, parecem OK
DataDocentesRaw['categoria'].unique()

In [ ]:
# Montando nosso data frame
docentesByIESDF = pd.DataFrame(columns=('IES', 'Permanentes', 'Colaboradores'))
docentesByIESDF['IES'] = DataDocentesRaw['ies'].unique()
docentesByIESDF['Permanentes'] = 0
docentesByIESDF['Colaboradores'] = 0

# Percorrendo o data frame original para fazer a contagem dos tipos de docente por instituição
for indx in range(0, len(DataDocentesRaw)):
    if DataDocentesRaw['categoria'][indx] == 'PERMANENTE':
        docentesByIESDF.loc[docentesByIESDF['IES'] == DataDocentesRaw['ies'][indx], 'Permanentes'] += 1
    elif DataDocentesRaw['categoria'][indx] == 'COLABORADOR':
        docentesByIESDF.loc[docentesByIESDF['IES'] == DataDocentesRaw['ies'][indx], 'Colaboradores'] += 1
    else: # Caso encontremos um imprevisto
        print("\'categoria\' error! Value:", DataDocentesRaw['categoria'][indx], " Row:", indx)
        
# O resultado obtido
docentesByIESDF

In [ ]:
# Montando um gráfico de barras horizontais empilhadas do resultado obtido
x_labels = docentesByIESDF['IES']
x = range(len(x_labels))
y = docentesByIESDF['Colaboradores']
x2 = []
for item in x:
    x2.append(item)
y2 = docentesByIESDF['Permanentes']

bar_width = 0.35

plt.figure()
plt.barh(x2, y2, height=bar_width, label="Permanentes", color='#4466cc')
plt.barh(x, y, height=bar_width, label ="Colaboradores", color='#cc6644')

plt.title('Professores x Programa')
plt.yticks(x, x_labels)
plt.legend(loc=4, frameon=True, title='Tipo de professor')

plt.show()

## 2) Quantos alunos (discentes) de Mestrado/Doutorado havia em cada programa em 2017?

In [ ]:
# DataDiscentesRaw contém os dados necessários para responder a pergunta (colunas "nivel" e "programa")
DataDiscentesRaw.head()

In [ ]:
# Pegando apenas as colunas que queremos
discentesDFQ2 = DataDiscentesRaw[['nivel','programa']]
display(discentesDFQ2.head()) # Vemos que temos uma coluna com valores vazios

display(discentesDFQ2.info()) # Vemos que 3 valores para cada coluna estão comprometidos

display(discentesDFQ2.loc[discentesDFQ2['nivel'].isnull()]) # Felizmente vemos que os 3 valores de cada coluna fazem parte das mesmas tuplas

# Limpando o data frame e mantendo a ordenação dos índices
discentesDFQ2 = discentesDFQ2.loc[discentesDFQ2['nivel'].isnull() == False]
discentesDFQ2 = discentesDFQ2.reset_index(drop = True)
discentesDFQ2.info()

In [ ]:
# Verificando valores em "nivel", vemos que existe pelo menos um valor 'Graduação' que não nos interessa
display(discentesDFQ2['nivel'].unique())

# Verificando valores em "programa"
display(discentesDFQ2['programa'].unique())

# Eliminando valores 'Graduação', obtemos uma redução considerável de tuplas
discentesDFQ2 = discentesDFQ2.loc[discentesDFQ2['nivel'] != 'Graduação']
discentesDFQ2 = discentesDFQ2.reset_index(drop = True)
discentesDFQ2.info()

In [ ]:
# Montando nosso data frame
discentesByPrograma = pd.DataFrame(columns=('Programa', 'Mestrandos', 'Doutorandos'))
discentesByPrograma['Programa'] = discentesDFQ2['programa'].unique()
discentesByPrograma['Mestrandos'] = 0
discentesByPrograma['Doutorandos'] = 0

# Percorrendo o data frame limpo para fazer a contagem dos tipos de nível por programa
for indx in range(0, len(discentesDFQ2)):
    if discentesDFQ2['nivel'][indx] == 'Mestrado':
        discentesByPrograma.loc[discentesByPrograma['Programa'] == discentesDFQ2['programa'][indx], 'Mestrandos'] += 1
    elif discentesDFQ2['nivel'][indx] == 'Doutorado':
        discentesByPrograma.loc[discentesByPrograma['Programa'] == discentesDFQ2['programa'][indx], 'Doutorandos'] += 1
    else: # Caso encontremos um imprevisto
        print("\'nivel\' error! Value:", discentesDFQ2['nivel'][indx], " Row:", indx)
        
# O resultado obtido
discentesByPrograma

In [ ]:
# Montando um gráfico de barras horizontais do resultado obtido
x_labels = discentesByPrograma['Programa']
x = range(len(x_labels))
y = discentesByPrograma['Mestrandos']

bar_width = 0.35

x2 = []
for item in x:
    x2.append(item + bar_width)
y2 = discentesByPrograma['Doutorandos']

plt.figure()
plt.barh(x, y, height=bar_width, label ="Mestrandos", color='#cc6644')
plt.barh(x2, y2, height=bar_width, label="Doutorandos", color='#4466cc')

plt.title('Alunos x Programa')
plt.yticks(x, x_labels)
plt.legend(loc=4, frameon=True, title='Tipo de aluno')

plt.show()

## 3) Qual foi a taxa de alunos de Mestrado/Doutorado por professor do quadro permanente em cada programa em 2017?

In [ ]:
# Vamos usar os data frames abaixo
display(DataDocentesRaw.head()) # Nos diz que professor é permanente
DataDiscentesRaw.head() # Nos diz qual o nível do aluno, o programa do qual fez parte e quem foi seu professor

In [ ]:
# Começando por DataDiscentesRaw
discentesDFQ3 = DataDiscentesRaw[['nivel','programa', 'orientador', 'orientadores']]
display(discentesDFQ3.info()) # Vemos que coluna 'orientador' tem muitos valores faltando e 'orientadores' tem menos, estranho
display(discentesDFQ3[0:1]) # A primeira linha nos mostra que existe um array em 'orientadores' mas este está vazio enquanto 'orientador' está com NaN, deve ser o que está causando a diferença numérica entre as colunas

# Retirando todas as colunas onde 'orientador' está nulo
discentesDFQ3 = discentesDFQ3.loc[discentesDFQ3['orientador'].isnull() == False]
discentesDFQ3 = discentesDFQ3.reset_index(drop = True)
display(discentesDFQ3.info()) # Agora todas as colunas possuem a mesma quantidade de linhas, originalmente sendo a de 'orientador' (o menor valor que tínhamos antes)

# Vemos que acabamos tirando o valor 'Graduação' em 'nivel' no meio dessa limpeza
display(discentesDFQ2['nivel'].unique())

discentesDFQ3.head()

In [ ]:
# Agora vamos com DataDocentesRaw
docentesDFQ3 = DataDocentesRaw[['categoria','nome']]
display(docentesDFQ3.info()) # Parece tudo OK

# Eliminando valores 'COLABORADOR'
docentesDFQ3 = docentesDFQ3.loc[docentesDFQ3['categoria'] != 'COLABORADOR']
docentesDFQ3 = docentesDFQ3.reset_index(drop = True)
display(docentesDFQ3.info()) # A redução não foi tão expressiva

docentesDFQ3.head()

In [ ]:
# Montando o data frame de resultado
taxasByPrograma = pd.DataFrame(columns=('Programa', 'Taxa Mestrandos/Professores', 'Taxa Doutorandos/Professores'))
taxasByPrograma['Programa'] = discentesDFQ3['programa'].unique()
taxasByPrograma['Taxa Mestrandos/Professores'] = 0
taxasByPrograma['Taxa Doutorandos/Professores'] = 0

# Procurando valores por valor 'programa'
for programa in taxasByPrograma['Programa']:
    # Reduzindo meu data frame 'discentes' para o valor 'programa' da iteração
    tempDiscentesDFQ3 = discentesDFQ3.loc[discentesDFQ3['programa'] == programa]
    
    # Reduzindo o data frame temporário ainda mais para valores 'orientador' com valor 'PERMANENTE'
    tempDiscentesDFQ3 = tempDiscentesDFQ3.loc[tempDiscentesDFQ3['orientador'].isin(docentesDFQ3['nome'])]
    tempDiscentesDFQ3 = tempDiscentesDFQ3.reset_index(drop = True)
    
    # Listas para guardar nomes do respectivo nivel do orientador
    orientadoresMestrado = []
    orientadoresDoutorado = []
    
    # Percorrendo o data frame temporário para fazer a contagem da quantidade de discentes por nível, guardando nome do docente
    for indx in range(0, len(tempDiscentesDFQ3)):
        if tempDiscentesDFQ3['nivel'][indx] == 'Mestrado':
            taxasByPrograma.loc[taxasByPrograma['Programa'] == programa, 'Taxa Mestrandos/Professores'] += 1
            if not orientadoresMestrado or tempDiscentesDFQ3['orientador'][indx] not in orientadoresMestrado:
                orientadoresMestrado.append(tempDiscentesDFQ3['orientador'][indx])
        elif tempDiscentesDFQ3['nivel'][indx] == 'Doutorado':
            taxasByPrograma.loc[taxasByPrograma['Programa'] == programa, 'Taxa Doutorandos/Professores'] += 1
            if not orientadoresDoutorado or tempDiscentesDFQ3['orientador'][indx] not in orientadoresDoutorado:
                orientadoresDoutorado.append(tempDiscentesDFQ3['orientador'][indx])
    
    # Calculando taxa "tipo discentes/docentes"
    if orientadoresMestrado:
        taxasByPrograma.loc[taxasByPrograma['Programa'] == programa, 'Taxa Mestrandos/Professores'] = taxasByPrograma.loc[taxasByPrograma['Programa'] == programa, 'Taxa Doutorandos/Professores'] / len(orientadoresMestrado)
    if orientadoresDoutorado:
        taxasByPrograma.loc[taxasByPrograma['Programa'] == programa, 'Taxa Doutorandos/Professores'] = taxasByPrograma.loc[taxasByPrograma['Programa'] == programa, 'Taxa Doutorandos/Professores'] / len(orientadoresDoutorado)

# O resultado obtido
taxasByPrograma

In [ ]:
# Montando resultado em gráfico de barras horizontais com o respectivo valor no final de cada barra
matplotlib.rcParams.update({'font.size': 10})

fig, ax = plt.subplots(figsize=(15,8))
y_pos = [i for i in range(0,len(taxasByPrograma['Programa']))]
x_values = list(taxasByPrograma['Taxa Mestrandos/Professores'])
x2_values = list(taxasByPrograma['Taxa Doutorandos/Professores'])
bar_width = 0.35
y2_pos = [i + bar_width for i in range(0,len(taxasByPrograma['Programa']))]

ax.set_title("Taxa \'tipo_alunos/professores\' x Programa")
ax.barh(y_pos, x_values, height=bar_width, label='Mestrandos', color='#cc6644')
ax.barh(y2_pos, x2_values, height=bar_width, label='Doutorandos', color='#4466cc')
ax.set_yticks(y_pos)
ax.set_yticklabels(list(taxasByPrograma['Programa']))
plt.xlim((0,max(x_values)*1.2))
rects = ax.patches

# For each bar: Place a label
for rect in rects:
    # Get X and Y placement of label from rect.
    x_value = rect.get_width()
    y_value = rect.get_y() + rect.get_height() / 2

    # Use Y value as label and format number with one decimal place
    label = "{:}".format(x_value)

    # Create annotation
    plt.annotate(
        label,                      # Use `label` as label
        (x_value, y_value),         # Place label at end of the bar
        xytext=(5, 5),              #  Shift label (horizontally,vertically)
        textcoords="offset points", # Interpret `xytext` as offset in points
        ha='left',                  # Horizontal label alignment
        va='top')                   # Vertical label alignment

plt.legend(loc=4, frameon=True, title='Tipo de aluno')    

plt.show()

## <span style="color:green">Desejável para a questão:</span>
- Confirmar que dados na coluna 'orientadores' tenham a mesma informação da sua respectiva tupla na coluna 'orientador' para justificar o uso de apenas 'orientador'. Ou seja, se existe só 1 orientador por aluno. (O nome 'orientadores' leva a crer que o aluno teve mais de um orientador no projeto)

## 4) Qual foi a distribuição de alunos de Mestrado/Doutorado pelos professores de cada programa em 2017?

## 5) Quantos alunos de Mestrado/Doutorado defenderam suas dissertações/teses em 2017 (arquivo trabalhos.csv)?

In [ ]:
# Vemos que faltam dados em DataTrabalhosRaw
display(DataTrabalhosRaw.info())
DataTrabalhosRaw.head()

In [ ]:
# Pegando as colunas que nos interessam e eliminando valores nulos através de uma das colunas do respectivo data frame
trabalhosDFQ5 = DataTrabalhosRaw.loc[DataTrabalhosRaw['autor'].isnull() == False][['autor', 'tipo', 'data_defesa']]
trabalhosDFQ5 = trabalhosDFQ5.sort_values(['autor', 'tipo'], ascending=[True, True])
trabalhosDFQ5 = trabalhosDFQ5.reset_index(drop = True)

display(trabalhosDFQ5.head())

# Vemos que felizmente nos desfizemos de todos os valores nulos. A coluna "data_defesa" foi deixada para nos certificarmos que
# todos os discentes aqui presentes realizaram as defesas
trabalhosDFQ5.info()

In [ ]:
# Em trabalhosDFQ5 temos apenas Tese e Dissertação, como queremos
display(trabalhosDFQ5['tipo'].unique())

In [ ]:
# Calculando o total de Mestrandos e Doutorandos
quantityMestrandos = len(trabalhosDFQ5.loc[trabalhosDFQ5['tipo'] == 'DISSERTAÇÃO'])
quantityDoutorandos = len(trabalhosDFQ5.loc[trabalhosDFQ5['tipo'] == 'TESE'])

print("Total Mestrandos:", quantityMestrandos)
print("Total Doutorandos:", quantityDoutorandos)

## **<span style="color:green">Desejável para a questão:</span>**
- Montar um gráfico de barras mostrando cada um dos valores. Uma barra para cada tipo.

## 6) Como os trabalhos de Mestrado/Doutorado defendidos em 2017 foram distribuídos pelas áreas de pesquisa dos programas?

# <span style="color:red">OBS.:</span>
Usar coluna tipo de tabela trabalhos para mestrado (tipo == 'DISSERTAÇÃO') e doutorado (tipo == 'TESE'). 

## 7) Como as defesas de Mestrado/Doutorado foram distribuídas ao longo do ano de 2017?

In [ ]:
# Para responder a pergunta, usaremos de novo 'DataTrabalhosRaw' (contém as datas) e 'DataDiscentesRaw' (contém o nivel)
(DataTrabalhosRaw.info()
display(DataTrabalhosRaw.head())

In [ ]:
# Pegando os atributos que nos interessa, fazendo as devidas limpezas/conversões
trabalhosDFQ7 = DataTrabalhosRaw.loc[(DataTrabalhosRaw['data_defesa'].isnull() == False)][['autor', 'tipo', 'data_defesa']]
trabalhosDFQ7['data_defesa'] = pd.to_datetime(trabalhosDFQ7['data_defesa'])
trabalhosDFQ7 = trabalhosDFQ7.sort_values(['data_defesa', 'tipo'], ascending=[True, True])
trabalhosDFQ7 = trabalhosDFQ7.reset_index(drop = True)

display(trabalhosDFQ7.info())
trabalhosDFQ7.head()

In [ ]:
# Montando nosso data frame
tipoByMes = pd.DataFrame(columns=('Mes', 'Mestrandos', 'Doutorandos'))
tipoByMes['Mes'] = trabalhosDFQ7['data_defesa'].dt.month.unique()
tipoByMes['Mestrandos'] = 0
tipoByMes['Doutorandos'] = 0

# Fazendo a contagem em relação ao tipo pra cada mês
for indx in range(0, len(trabalhosDFQ7)):
    if trabalhosDFQ7['tipo'][indx] == 'DISSERTAÇÃO':
        tipoByMes.loc[tipoByMes['Mes'] == trabalhosDFQ7['data_defesa'].dt.month[indx], 'Mestrandos'] += 1
    elif trabalhosDFQ7['tipo'][indx] == 'TESE':
        tipoByMes.loc[tipoByMes['Mes'] == trabalhosDFQ7['data_defesa'].dt.month[indx], 'Doutorandos'] += 1

# Nosso data frame resultante
tipoByMes

## **<span style="color:green">Desejável para a questão:</span>**
- Montar um "gráfico de linhas" mostrando a evolução de cada tipo ao longo dos meses. Uma linha para cada tipo (mestrado e doutorado)

## 8) Qual fração de alunos de Mestrado/Doutorado (do total em cada programa) defendeu em 2017?

## 9) Quantos artigos de periódico/trabalhos em anais foram publicados por cada programa em 2017?

In [ ]:
# Usaremos tabela 'produção' para responder a pergunta
DataProducaoRaw.info()
display(DataProducaoRaw.head())

In [ ]:
# Pegando apenas as colunas que nos interessam, temos todos os programas, mas há artigos e trabalhos como nulo
producaoDFQ9 = DataProducaoRaw[['programa', 'paper_id', 'periodico', 'anais_titulo']]
producaoDFQ9 = producaoDFQ9.drop_duplicates(subset='paper_id', keep=False) # Nos certificando de retirar papers duplicados
producaoDFQ9 = producaoDFQ9.reset_index(drop = True)
display(producaoDFQ9.info())
producaoDFQ9.head()

In [ ]:
# Montando nosso data frame
artigosTrabalhosByPrograma = pd.DataFrame(columns=('Programa', 'Artigos de periódico', 'Trabalhos em anais'))
artigosTrabalhosByPrograma['Programa'] = producaoDFQ9['programa'].unique()
artigosTrabalhosByPrograma['Artigos de periódico'] = 0
artigosTrabalhosByPrograma['Trabalhos em anais'] = 0

# Fazendo a contagem em relação a artigos e trabalhos para o respectivo programa
for indx in range(0, len(producaoDFQ9)):
    if producaoDFQ9['periodico'][indx] is not np.nan:
        artigosTrabalhosByPrograma.loc[artigosTrabalhosByPrograma['Programa'] == producaoDFQ9['programa'][indx], 'Artigos de periódico'] += 1
    if producaoDFQ9['anais_titulo'][indx] is not np.nan:
        artigosTrabalhosByPrograma.loc[artigosTrabalhosByPrograma['Programa'] == producaoDFQ9['programa'][indx], 'Trabalhos em anais'] += 1

# Nosso data frame resultante
artigosTrabalhosByPrograma

## **<span style="color:green">Desejável para a questão:</span>**
- Montar um gráfico de barras horizontais por programa. Cada programa tendo uma dupla  de barras (artigos e trabalhos)

## 10) Qual é a taxa de artigos de periódico/trabalhos em anais por número de docentes permanentes de cada programa em 2017?

## 11) Qual fração de artigos de periódico/trabalhos em anais publicados em 2017 teve a coautoria de discentes?

In [ ]:
# Usaremos tabela 'produção' para responder a pergunta
DataProducaoRaw.info()
display(DataProducaoRaw.head())

In [ ]:
# Pegando as colunas que nos interessam
producaoDFQ11 = DataProducaoRaw[['num_docentes', 'num_discentes', 'num_externos', 'paper_id', 'periodico', 'anais_titulo']]
producaoDFQ11 = producaoDFQ11.drop_duplicates(subset='paper_id', keep=False) # Retirando os papers duplicados
producaoDFQ11 = producaoDFQ11.reset_index(drop = True)
display(producaoDFQ11.info())
producaoDFQ11.head()

In [ ]:
# Existe um valor sem docente e externo. Como queremos coautoria com discentes, precisamos de pelo menos um participante para
# pelo menos um deles. Logo, retiremos essa tupla.
display(producaoDFQ11.loc[(producaoDFQ11['num_docentes'] == 0) & (producaoDFQ11['num_externos'] == 0)])

producaoDFQ11 = producaoDFQ11.drop(producaoDFQ11.index[[2556]])
producaoDFQ11 = producaoDFQ11.reset_index(drop = True)

In [ ]:
# Declarando e inicializando as variáveis que usaremos
artigosDiscentePercent = 0
trabalhosDiscentePercent = 0
artigosTotal = 0
trabalhosTotal = 0

# Fazendo a contagem do total de artigos e trabalhos e de coautoria de discente para cada um deles
for indx in range(0, len(producaoDFQ11)):
    if producaoDFQ11['periodico'][indx] is not np.nan:
        artigosTotal += 1
        if producaoDFQ11['num_discentes'][indx] != 0:
            artigosDiscentePercent += 1
    if producaoDFQ11['anais_titulo'][indx] is not np.nan:
        trabalhosTotal += 1
        if producaoDFQ11['num_discentes'][indx] != 0:
            trabalhosDiscentePercent += 1

# Calculando as porcentagens
artigosDiscentePercent /= artigosTotal
artigosDiscentePercent *= 100
trabalhosDiscentePercent /= trabalhosTotal
trabalhosDiscentePercent *= 100

print("\nArtigos de periódico com coautoria de discente: ", artigosDiscentePercent, "%\nTrabalhos em anais com coautoria de discente: ", trabalhosDiscentePercent, "%")

## **<span style="color:green">Desejável para a questão:</span>**
- Montar um gráfico de pizza para mostrar as frações obtidas em comparação com o total do respectivo produto publicado. Ou seja, um gráfico para frações de artigos com e sem coautoria de discente e outro gráfico para frações de trabalhos com e sem coautoria de discente.

## 12) Qual fração de artigos de periódico/trabalhos em anais publicados em 2017 teve a coautoria de participantes externos?

## 13) Qual é a distribuição de artigos de periódico publicados em 2017, por estrato do Qualis?

## 14) Considerando os pesos dos artigos de cada estrato do índice restrito ({'A1': 1, 'A2': 0.85, 'B1': 0.70}), qual o índice restrito relativo de cada programa, considerando apenas os artigos em periódicos, e dividido pelo número de docentes permanentes (ordenado do maior para o menor)?

In [ ]:
# Para responder a questão, usaremos tabela 'qualis' que nos diz o ISSN e o Extrato
display(DataQualisRaw.info())
display(DataQualisRaw.head())

# Tabela 'produção' que nos diz o programa, o ISSN, se produto é artigo em periódico e os docentes que participaram
display(DataProducaoRaw.info())
display(DataProducaoRaw.head(4))

# Tabela 'docentes' que nos diz se docente é permanente
display(DataDocentesRaw.info())
DataDocentesRaw.head()

In [ ]:
# Limpando meus data frames e pegando apenas os dados que nos interessam
qualisDFQ14 = DataQualisRaw.drop_duplicates(subset=['ISSN', 'Título', 'Estrato'], keep=False)[['ISSN', 'Estrato']]
qualisDFQ14.columns.values[0] = "issn"
qualisDFQ14.columns.values[1] = "estrato"
display(qualisDFQ14.info())
display(qualisDFQ14.head())

producaoDFQ14 = DataProducaoRaw.loc[(DataProducaoRaw['periodico'].isnull() == False)
                                   & (DataProducaoRaw['num_docentes'] != 0)][['programa', 'paper_id', 'dict_paper_autores', 'issn']]
producaoDFQ14 = producaoDFQ14.drop_duplicates(subset='paper_id', keep=False)
producaoDFQ14 = producaoDFQ14.reset_index(drop = True)
producaoDFQ14 = producaoDFQ14.drop('paper_id', 1)
display(producaoDFQ14.info())
display(producaoDFQ14.head())

docentesDFQ14 = DataDocentesRaw.loc[DataDocentesRaw['categoria'] == 'PERMANENTE'][['nome']]
display(docentesDFQ14.info())
docentesDFQ14.head()

In [ ]:
# Montando nosso data frame resultado
indiceRestritoRelativoByPrograma = pd.DataFrame(columns=('Programa', 'Índice restrito relativo', 'Total docentes permanentes'))
indiceRestritoRelativoByPrograma['Programa'] = producaoDFQ14['programa'].unique()
indiceRestritoRelativoByPrograma['Índice restrito relativo'] = 0
indiceRestritoRelativoByPrograma['Total docentes permanentes'] = 0

# Criando dicionário para os valores dos estratos que restringem nosso cálculo
estratoDictionary = {'A1': 1, 'A2': 0.85, 'B1': 0.70}

# Fazendo a contagem do índice relativo em relação ao programa e se há docente que é permanente
for indx in range(0, len(producaoDFQ14)):
    dictionaryList = ast.literal_eval(producaoDFQ14['dict_paper_autores'][indx])
    for dictionary in dictionaryList:
        if (dictionary['categoria'] == 'Docente') & (docentesDFQ14[docentesDFQ14['nome'].str.contains(dictionary['nome'])].empty == False) & (qualisDFQ14[qualisDFQ14['issn'].str.contains(producaoDFQ14['issn'][indx])].empty == False):
            indiceRestritoRelativoByPrograma.loc[indiceRestritoRelativoByPrograma['Programa'] == producaoDFQ14['programa'][indx], 'Total docentes permanentes'] += 1
                    
            estrato = qualisDFQ14.loc[qualisDFQ14['issn'] == producaoDFQ14['issn'][indx], "estrato"].reset_index(drop = True)[0]
            if estrato in estratoDictionary:
                indiceRestritoRelativoByPrograma.loc[indiceRestritoRelativoByPrograma['Programa'] == producaoDFQ14['programa'][indx], 'Índice restrito relativo'] += estratoDictionary[estrato]
            break

# Calculando com o total de docentes permanentes do respectivo programa
indiceRestritoRelativoByPrograma['Resultado'] = indiceRestritoRelativoByPrograma['Índice restrito relativo'] / indiceRestritoRelativoByPrograma['Total docentes permanentes']

# Nosso data frame resultante
indiceRestritoRelativoByPrograma = indiceRestritoRelativoByPrograma.sort_values(['Resultado', 'Índice restrito relativo'], ascending=[False, False])
indiceRestritoRelativoByPrograma = indiceRestritoRelativoByPrograma.reset_index(drop = True)
indiceRestritoRelativoByPrograma

## **<span style="color:green">Desejável para a questão:</span>**
- Montar um gráfico de barras mostrando a ordem decrescente do resultado obtido, uma barra pra cada programa.
- Otimizar o código para obter resultado, está demorando bastante (dá pra eliminar tuplas de 'produção' checando na limpeza se um docente não é permanente, por exemplo)

## 15) Considerando os pesos dos artigos de cada estrato do índice geral ({'A1': 1, 'A2': 0.85, 'B1': 0.70, 'B2': 0.50, 'B3': 0.20, 'B4': 0.10, 'B5': 0.05, 'C': 0.0}), qual o índice geral relativo de cada programa, considerando apenas os artigos em periódicos, e dividido pelo número de docentes permanentes (ordenado do maior para o menor)?

## **<span style="color:red">Detalhes a ficarmos atentos para o trabalho:</span>**
- Verificar repetição de dados (aparentemente temos papers duplicados)
- Estar atento a outros detalhes sobre os dados não percebidos na análise inicial dos dados